# Ave, CELEBA! For Latent space with deterministic maps

In [ ]:
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
import itertools
import wandb
import os

import sys
sys.path.append("../../")
from src.utils import Config, weights_init_D, freeze, unfreeze, normalize_out_to_0_1
from src.data import DatasetSampler
from src.models import ResNet_D, UNet, linear_model
from src.train import train
from src.cost import strong_cost

# for generator
import sys 
sys.path.append("../../stylegan2/")
import dnnlib
import legacy

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

## 1. Config

In [ ]:
BATCH_SIZE = 64
NUM_EPOCHS = 1
INNER_ITERATIONS = 10
GPU_DEVICE = 0
FLAG_LATENT = True
FLAG_LATENT_CRITIC = False
LR_MAP = 1e-4
LR_POTENTIAL = 1e-4
CHI = False
SPHERE = True

In [ ]:
CONFIG = Config()

CONFIG.FLAG_LATENT =  FLAG_LATENT
CONFIG.FLAG_LATENT_CRITIC = FLAG_LATENT_CRITIC 
CONFIG.NC_LATENT = 1
CONFIG.LATENT_SIZE = 512
CONFIG.GENERATOR_PATH =  "../../../stylegan2_ada_pytorch_before/training-runs/00011-aligned_celeba-stylegan2/network-snapshot-008800.pkl"
CONFIG.N_LATENT_RESNET=512
CONFIG.CHI_PROJECTION = CHI
CONFIG.SPHERE_PROJECTION = SPHERE
CONFIG.RESNET_D_ENCODER = False

CONFIG.K = 3# amount of distributions
CONFIG.IMG_SIZE = 64
CONFIG.NC=3
CONFIG.BASE_FACTOR = 64


CONFIG.LR_MAP = LR_MAP
CONFIG.LR_POTENTIAL = LR_POTENTIAL
CONFIG.CLIP_GRADS_NORM = False
CONFIG.BETAS = (0.2, 0.99)

CONFIG.HIDDEN_SIZE = [max(2*CONFIG.N_LATENT_RESNET,128),
                      max(2*CONFIG.N_LATENT_RESNET,128)] 

CONFIG.CLASSES = [0,1,2]
CONFIG.DATASET_PATH ='../../../data/ave_celeba_green_v2/' 
CONFIG.DATASET = 'ave_celeba'

CONFIG.LAMBDAS = [0.25,0.5,0.25]
CONFIG.DIM = 3*64*64
CONFIG.BATCH_SIZE = BATCH_SIZE
CONFIG.NUM_EPOCHS = NUM_EPOCHS
CONFIG.INNER_ITERATIONS = INNER_ITERATIONS

CONFIG.GPU_DEVICE = GPU_DEVICE
assert torch.cuda.is_available()
CONFIG.DEVICE = f'cuda:{CONFIG.GPU_DEVICE}'

## 2. Generator

In [ ]:
if CONFIG.FLAG_LATENT:
    with dnnlib.util.open_url(CONFIG.GENERATOR_PATH) as f:
        G =  legacy.load_network_pkl(f)['G_ema'].to(CONFIG.DEVICE)

## 3. Potential and maps

### 3.1 Potentials

In [ ]:
if CONFIG.K == 2:
    
    f = ResNet_D(size=CONFIG.IMG_SIZE,
              nc=CONFIG.NC,
              nfilter=64, 
              nfilter_max=512, 
              res_ratio=0.1,bn_flag=False,pn_flag=False).to(CONFIG.DEVICE)
    
    weights_init_D(f)
    nets_for_pot = [f]
     
    
    nets_for_pot_opt = torch.optim.Adam( nets_for_pot[0].parameters() ,
                               CONFIG.LR_POTENTIAL, betas=CONFIG.BETAS)
                   
else: 
    
    if CONFIG.FLAG_LATENT_CRITIC:
        
        nets_for_pot = [ linear_model(CONFIG.LATENT_SIZE, CONFIG.HIDDEN_SIZE, 1).to(CONFIG.DEVICE)
                        for k in range(CONFIG.K)   
        ]
        
    else:
        nets_for_pot = [ResNet_D(size=CONFIG.IMG_SIZE,
                  nc=CONFIG.NC,
                  nfilter=64, 
                  nfilter_max=512, 
                  res_ratio=0.1,n_output=1,bn_flag=False,pn_flag=False).to(CONFIG.DEVICE)
                for i in range(CONFIG.K)]

        for f in nets_for_pot: 
            weights_init_D(f)
    
    param_nets = [net.parameters() for net in nets_for_pot]
    nets_for_pot_opt = torch.optim.Adam(itertools.chain(*param_nets),
                               CONFIG.LR_POTENTIAL, betas=CONFIG.BETAS)
 

### 3.2 maps

In [ ]:
if CONFIG.FLAG_LATENT:
    
 
    maps = [ResNet_D(size=CONFIG.IMG_SIZE,
              nc=CONFIG.NC,
              nfilter=64, 
              nfilter_max=512, 
              res_ratio=0.1,n_output=CONFIG.N_LATENT_RESNET,bn_flag=True,pn_flag=True).to(CONFIG.DEVICE) for i in range(CONFIG.K)]
    
    for k in range(CONFIG.K):
        weights_init_D(maps[k])
else:
    
    maps = [UNet(n_channels=CONFIG.NC, n_classes=CONFIG.NC,
                 base_factor=48 , bilinear=True).to(CONFIG.DEVICE) for i in range(CONFIG.K)]

param_maps = [mp.parameters() for mp in maps]
maps_opt = torch.optim.Adam(itertools.chain(*param_maps),
                               CONFIG.LR_MAP, betas=CONFIG.BETAS)

## 4. Data Samplers

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(CONFIG.IMG_SIZE),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Lambda(lambda x: torch.clip(x,0,1))
])

data_samplers=[]
for k in tqdm(range(CONFIG.K)):
    dataset = torchvision.datasets.ImageFolder(os.path.join(CONFIG.DATASET_PATH,f"ave_celeba_{k}/"), transform=transform)
    data_samplers.append(DatasetSampler(dataset, flag_label=True, batch_size=256 ,num_workers=40))
    
    

## 5. WandB

In [ ]:
name_exp = f"LATENT_BS_{BATCH_SIZE}_INNER_{INNER_ITERATIONS}"
CONFIG.NAME_EXP = name_exp

In [ ]:
wandb.init(project="BNOT" ,
           name=name_exp,
           config=CONFIG)

## 5. Train

In [ ]:
train(nets_for_pot, 
          maps,
          nets_for_pot_opt,
          maps_opt,
          data_samplers,
          G,
          CONFIG)